In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [2]:
data = pd.read_csv('/home/smansou2/Desktop/experimentAnalysis/scdb-test-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])
print(data.head())

               Event  Time Tx_type  \
0        received_tx  2869  CREATE   
1  before_tendermint  2969  CREATE   
2           check_tx  2985  CREATE   
3         deliver_tx  3011  CREATE   
4          end_block  3013  CREATE   

                                               Tx_id Accept_id          time  
0  5744711a042889d11a016a1268bb2b9e9cd53cc32fa64d...      None  1.733811e+09  
1  5744711a042889d11a016a1268bb2b9e9cd53cc32fa64d...      None  1.733811e+09  
2  5744711a042889d11a016a1268bb2b9e9cd53cc32fa64d...      None  1.733811e+09  
3  5744711a042889d11a016a1268bb2b9e9cd53cc32fa64d...      None  1.733811e+09  
4  5744711a042889d11a016a1268bb2b9e9cd53cc32fa64d...      None  1.733811e+09  


In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

5834

In [4]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

create_count = 0
create_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'CREATE':
        create_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        create_count += 1   


print(f"#CREATEs =  {create_count}")

#CREATEs =  2704


In [5]:
df_test = grouped_data.get_group("30dde698483f4f5c17186647a54cf21f2bfc745f4afb68a08bfeda867cbf7d1d")

KeyError: '30dde698483f4f5c17186647a54cf21f2bfc745f4afb68a08bfeda867cbf7d1d'

In [ ]:
df_test

NameError: name 'df_test' is not defined

In [6]:
time_all_committed_creates = {}
time_passed_creates = []
validated_time = []
accepted_time = []
committed_time = []

create_underfit = []
create_underfit_dict = {}

create_overfit = []
create_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "CREATE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_creates[tx_id[i]] = actual_time
            time_passed_creates.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            create_underfit.append(temp_list)
            create_underfit_dict[tx_id(i)] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_creates[tx_id[i]] = actual_time
            time_passed_creates.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            create_overfit.append(temp_list)
            create_overfit_dict[tx_id[i]] = temp_list

TypeError: 'list' object is not callable

In [7]:
# Analysis of CREATE Transaction
create_mean = "{:.2f}".format(stat.mean(time_passed_creates))
create_median = stat.median(time_passed_creates)
create_max = max(time_passed_creates)
create_min = min(time_passed_creates)
create_std_dev = "{:.2f}".format(stat.stdev(time_passed_creates))

print(create_mean)
print(create_median)
print(create_max)
print(create_min)
print(create_std_dev)
print(f"total creates = {len(time_passed_creates)}")

create_dict = {
               'min': create_min,
               'max': create_max,
               'average': create_mean,
               'median' : create_median,
               'standard_deviation' : create_std_dev,
               '# of txs' : len(time_passed_creates)}

14283.53
56
2431555
41
185940.74
total creates = 171


In [8]:
create_overfit_dict

{'004ee743090d8916fba66d39bfe74945ca14099110822e0ba62892ab36f99c2e': [135,
  142,
  155,
  197,
  251,
  259,
  270],
 '0ead4d7339e3b96d567feec3e9cb58b495cd2305e1edeaa68e0af2a771eb61f0': [165,
  174,
  200,
  219,
  283,
  286,
  294],
 '0f1bebb13cfcfd61af6d938c64eb657c929017029242fe4b38ba99a5271b67c5': [53,
  61,
  74,
  156,
  189,
  196,
  204]}

In [9]:
time_all_committed_creates

{'000ca0facc38fd65452d1901b2b1e78688afb255ab1e3b1ea4bfd1ad56ace910': 100,
 '000ee0ea0e590f69851632445c6bce32d6f1ae3e9c8900685708f6de9f69e634': 46,
 '00300f43e404541bbfb7ce696513b480e6fbfb0afdf2a0eea33e512bc6fc8383': 50,
 '003a499dbe24e464481ed63a137b47a574cbf37e012e8d6a8ee6a8ef202a8907': 51,
 '0047ceb4beb5a750c2a4216625242581e0171417ea2f03e60ee623db4f7a777d': 56,
 '004ee743090d8916fba66d39bfe74945ca14099110822e0ba62892ab36f99c2e': 135,
 '006d96c18e7a0b852886e5047ee87aeaded663fd917a5b4827eb8cee68e702e6': 48,
 '00873e6be6b94b6fca81a00ba02b4f09e8a95e5b005294e4dcbae67ec2901e11': 56,
 '009c2f5bf8dcc235da72dc8221e246695c2f688b4078b390785fcd6d3b1b686f': 80,
 '00bf844c7c723a2d34e6103098d13ac494127c75f0274633a9929d14bddb2d83': 49,
 '00c3340749184e93b8e835bccfd666afa7530a0274542b3e58a375e46da8add3': 50,
 '00d62734513af35352f45f93355e136dcbf9f737325417ae5a06fb2d3d77c220': 61,
 '010bafe66b767db833ca1e7af836c851fc7990014fff73be5f5181b83bf3c1e6': 51,
 '010e57408d0d36793a70fd57ef681896f55581ae4bf86b3

In [10]:
results = pd.DataFrame(create_dict, index = ['CREATE'])
results.to_csv('create_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
CREATE,41,2431555,14283.53,56,185940.74,171
